In [1]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool


import dotenv
dotenv.load_dotenv()

True

In [20]:
#llm = create_connector_llm(
#    provider='llama_cpp',
#    modelname='not-needed',
#    version='not-needed',
#    credentials=CredentialsOpenAI(
#        base_url='http://llm-server:8080/v1',
#        api_key='not-needed',
#    ),
#    hyperparameters = {'max_tokens': 10}
#)

llm = create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
)

max_steps_recommended = 1

orchestrator = OrchestratorBare(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
    tool_callables = generate_callables([(PluginCapital(), "PluginCapital")]),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = max_steps_recommended,
    max_steps_allowed = 3,
    system_prompt=f'''you are an AI assistant whose goal is to answer the user question. 
    Always use the tools, do not answer without the tools.
    Your decisions must always be made independently without seeking user assistance.
    Every command has a cost, so be smart and efficient. You'll be rewarded if you answer correctly and quickly.
    Aim to complete tasks in the least number of steps. Whenever you are ready to answer, use the function 'answer'.
    Do not take more than {max_steps_recommended} steps to complete a task (including the function 'answer').
    If at step {max_steps_recommended - 1} you don't have all necessary information, answer the best you can with the information at hand.
    Always try to answer with as much useful information as you can.
    ''',
    triggering_prompt='''Determine which next function to use next.
    If you have enough information to answer the question, use the 'answer' function to signal and remember show your results.
    Call only one function per message.
    Do not call the same function with the same arguments more than once, their outputs are deterministic.
    ''',
    user_prompt='Question: What is the capital of brasil?',
)
r = await orchestrator.run()
print(r)
assert type(r.answer) == str
assert len(r.answer) > 4

In [21]:
r = await orchestrator.run()
print(r)
assert type(r.answer) == str
assert len(r.answer) > 4

[2024-08-09 14:03:42 +0000] libs.libs.plugin_orchestrator INFO     At step 1 the function PluginCapital_get_capital was requested, with args = {'country': 'germany'}
[2024-08-09 14:03:42 +0000] libs.libs.plugin_orchestrator INFO     result: Executed function PluginCapital_get_capital and returned: berlin
As this was the step 1/1, next you should call 'answer' with all the results you have.
[2024-08-09 14:03:43 +0000] libs.libs.plugin_orchestrator INFO     At step 2 the function answer was requested, with args = {'text': 'The capital of Germany is Berlin.'}


ValidatedAnswer(answer='The capital of Germany is Berlin.', citations=[Citation(id='bacfb720', document='Result of function PluginCapital_get_capital', content="Executed function PluginCapital_get_capital and returned: berlin\nAs this was the step 1/1, next you should call 'answer' with all the results you have.")], visualizations=[])

'The capital of Brazil is Brasilia.'

True

In [20]:
llm = create_connector_llm(
    provider='llama_cpp',
    modelname='not-needed',
    version='not-needed',
    credentials=CredentialsOpenAI(
        base_url='http://llm-server:8080/v1',
        api_key='not-needed',
    ),
    hyperparameters = {'max_tokens': 10}
)


In [21]:
llm.capabilities

Capabilities(response_json_only=False, tool_call=False, local=False)

In [2]:
llm = create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
)

In [3]:
# chat completion, normal, with function call
chat_completion = llm.chat_completion(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
print(chat_completion.choices[0].message.tool_calls[0].dict())
assert len(chat_completion.choices[0].message.tool_calls) > 0
assert 'PluginCapital' in chat_completion.choices[0].message.tool_calls[0].function.name

{'id': 'call_8PORR24cJe5zII4KFyYMgo7c', 'type': 'function', 'function': {'arguments': '{"country":"Brazil"}', 'name': 'PluginCapital_get_capital'}}


In [14]:
# chat completion, stream, with function call
chat_completion_stream = llm.chat_completion_stream(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
name=None
arguments=''
for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        print(chunk.choices[0].message, end='\n-----\n')
        if chunk.choices[0].message.tool_calls and len(chunk.choices[0].message.tool_calls) > 0:
            if (name is None) and (chunk.choices[0].message.tool_calls[0].function.name is not None):
                name = chunk.choices[0].message.tool_calls[0].function.name
            if chunk.choices[0].message.tool_calls[0].function.arguments is not None:
                arguments += chunk.choices[0].message.tool_calls[0].function.arguments
assert 'PluginCapital' in name
assert 'brazil' in arguments.lower()

content='' role='assistant' tool_calls=[OpenAIToolCallPart(id='call_FAaJ1NzQjG0xZh0Znx1xfdDA', type='function', function=OpenaiFunctionCallPart(arguments='', name='PluginCapital_get_capital'))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='{"', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='country', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='":"', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='Brazil', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='"}', name=None))]
-----
content='' role='assistant' tool_calls=None
-----


In [18]:
# chat completion, normal, async
chat_completion = await llm.chat_completion_async(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
print(chat_completion.choices[0].message.tool_calls[0].dict())
assert len(chat_completion.choices[0].message.tool_calls) > 0
assert 'PluginCapital' in chat_completion.choices[0].message.tool_calls[0].function.name

{'id': 'call_2xZg8Wswa7OnLzF4soFxm2dM', 'type': 'function', 'function': {'arguments': '{"country":"brasil"}', 'name': 'PluginCapital_get_capital'}}


In [19]:
# chat completion, stream, async
chat_completion_stream = llm.chat_completion_stream_async(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)

name=None
arguments=''
async for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        print(chunk.choices[0].message, end='\n-----\n')
        if chunk.choices[0].message.tool_calls and len(chunk.choices[0].message.tool_calls) > 0:
            if (name is None) and (chunk.choices[0].message.tool_calls[0].function.name is not None):
                name = chunk.choices[0].message.tool_calls[0].function.name
            if chunk.choices[0].message.tool_calls[0].function.arguments is not None:
                arguments += chunk.choices[0].message.tool_calls[0].function.arguments
assert 'PluginCapital' in name
assert 'brazil' in arguments.lower()

content='' role='assistant' tool_calls=[OpenAIToolCallPart(id='call_I4a3D0Td4famiQt7CwVHy9mw', type='function', function=OpenaiFunctionCallPart(arguments='', name='PluginCapital_get_capital'))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='{"', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='country', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='":"', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='Brazil', name=None))]
-----
content='' role='assistant' tool_calls=[OpenAIToolCallPart(id=None, type=None, function=OpenaiFunctionCallPart(arguments='"}', name=None))]
-----
content='' role='assistant' tool_calls=None
-----


In [16]:
arguments

'{"country":"Brazil"}'

In [12]:
chunk.choices[0].message.tool_calls[0].function.arguments

''

True

In [3]:
# text completion, normal
chat_completion = llm.chat_completion(
    messages=[ChatCompletionMessage(content='hi', role='user')],
)
assert type(chat_completion.choices[0].message.content) == str
assert len(chat_completion.choices[0].message.content) > 2
print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
# chat completion, stream
chat_completion_stream = llm.chat_completion_stream(
    messages=[ChatCompletionMessage(content='hi', role='user')],
)
answer = ''
for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        assert type(chunk.choices[0].message.content) == str
        print(chunk.choices[0].message.content, end='')
        answer += chunk.choices[0].message.content
assert len(answer) > 2

Hello! How can I help you today?

In [5]:
# text completion, normal, async
chat_completion = await llm.chat_completion_async(
    messages=[ChatCompletionMessage(content='hi', role='user')],
)
assert type(chat_completion.choices[0].message.content) == str
assert len(chat_completion.choices[0].message.content) > 2
print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


In [6]:
# chat completion, stream, async
chat_completion_stream = llm.chat_completion_stream_async(
    messages=[ChatCompletionMessage(content='hi', role='user')],
)
answer = ''
async for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        assert type(chunk.choices[0].message.content) == str
        print(chunk.choices[0].message.content, end='')
        answer += chunk.choices[0].message.content
assert len(answer) > 2

Hello! How can I assist you today?

In [8]:
llm = create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
)

In [10]:
llm = create_connector_llm(
    provider='llama_cpp',
    modelname='not-needed',
    version='not-needed',
    credentials=CredentialsOpenAI(
        base_url='http://llm-server:8080/v1',
        api_key='not-needed',
    ),
)


In [11]:
# text completion, normal
chat_completion = llm.chat_completion(
    messages=[ChatCompletionMessage(content='hi', role='user')],
)
chat_completion.choices[0].message.content

TypeError: ConnectorLLMOpenAI.chat_completion() missing 1 required positional argument: 'tool_definitions'

In [9]:
# chat completion, normal, with function call
chat_completion = llm.chat_completion(
    messages=[
        #ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
print(chat_completion.choices[0].message.content)
print('----')
print(chat_completion.choices[0].message.tool_calls[0].dict())

None
----
{'id': 'call_u3Xr6nTOpkPZPviCkrR1h7Zq', 'type': 'function', 'function': {'arguments': '{"country": "Brazil"}', 'name': 'PluginCapital_get_capital'}}


In [6]:
# chat completion, stream, with function call
chat_completion_stream = llm.chat_completion_stream(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        #print(chunk.choices[0].message.content, end='')
        print(chunk.choices[0].message, end='\n-----\n')

InternalServerError: Error code: 500 - {'error': {'code': 500, 'message': 'Unsupported param: tools', 'type': 'server_error'}}

In [ ]:
# chat completion, normal, async
chat_completion = await llm.chat_completion_async(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
print(chat_completion.choices[0].message.content)
print('----')
print(chat_completion.choices[0].message.tool_calls[0].dict())

In [ ]:
# chat completion, stream, async
chat_completion_stream = llm.chat_completion_stream_async(
    messages=[
        ChatCompletionMessage(**{'role': 'system', 'content': 'you are an AI assistant that answer only with function calls, nothing else. Always use the tools, do not answer without the tools.'}),
        ChatCompletionMessage(**{'role': 'user', 'content': 'What is the capital of brasil?'}),
    ],
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
)
async for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        #print(chunk.choices[0].message.content, end='')
        print(chunk.choices[0].message, end='\n-----\n')

In [ ]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [ ]:
chat_completion = await client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    messages= [
    {
      "role": "user",
      "content": "What are the differences between Azure Machine Learning and Azure AI services?"
    }],
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)
print(type(chat_completion))
#print(chat_completion_stream.to_dict().keys())
#print(chat_completion_stream.to_json())

In [ ]:
chat_completion.to_json()

In [ ]:
print(type(chat_completion))


In [ ]:
dir(chat_completion_stream)

In [ ]:
chat_completion_stream.to_json()

In [ ]:
print(type(chat_completion_stream))
#print(chat_completion_stream.to_dict().keys())
#print(chat_completion_stream.to_json())

In [ ]:
chat_completion_stream = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    messages= [
    {
      "role": "user",
      "content": "What are the differences between Azure Machine Learning and Azure AI services?"
    }],
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=True
)
print(type(chat_completion_stream))
#print(chat_completion_stream.to_dict().keys())
#print(chat_completion_stream.to_json())

In [ ]:
type(chat_completion_stream._iterator)

In [ ]:
chunk = chat_completion_stream.__next__()

In [ ]:
type(chunk)

In [ ]:
chunk.dict()

In [ ]:
chunk.choices[0].delta.dict()

In [ ]:
ChatCompletion(**chunk.dict())

In [ ]:
dir(chat_completion_stream)

In [ ]:
print(chat_completion_stream.to_dict().keys())

In [ ]:
ChatCompletion(**chat_completion.dict()).dict()